In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
from gtsam.symbol_shorthand import X
import matplotlib.pyplot as plt

from lac.slam.feature_tracker import FeatureTracker
from lac.utils.plotting import plot_poses, plot_surface, plot_3d_points
from lac.util import load_data, load_stereo_images
from lac.params import LAC_BASE_PATH

%load_ext autoreload
%autoreload 2

In [ ]:
# Load the data logs
data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/stereo_lights1.0_map1_preset1"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

# Load the images
left_imgs, right_imgs = load_stereo_images(data_path)

# Load the ground truth map
map = np.load(
    Path(LAC_BASE_PATH) / "data/heightmaps/competition/Moon_Map_01_preset_0.dat",
    allow_pickle=True,
)

In [ ]:
plot_poses(poses, no_axes=True, color="black", name="Ground Truth")

In [ ]:
tracker = FeatureTracker(cam_config)

In [ ]:
# Select 2 images from similar viewpoints
frame1 = 600
frame2 = 4300
img1 = left_imgs[frame1]
img2 = left_imgs[frame2]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(img1, cmap="gray")
ax[1].imshow(img2, cmap="gray")
ax[0].axis("off")
ax[1].axis("off")
plt.show()

In [ ]:
feats1 = tracker.extract_feats(img1)
feats2 = tracker.extract_feats(img2)
matches = tracker.match_feats(feats1, feats2)

points1 = feats1["keypoints"][0][matches[:, 0]].cpu().numpy()
points2 = feats2["keypoints"][0][matches[:, 1]].cpu().numpy()

In [ ]:
from lightglue import viz2d

viz2d.plot_images([img1, img2])
viz2d.plot_matches(points1, points2, lw=0.2)

# Loop closure factors

Assuming the loop closure has been detected:

1. Stereo matching: For each frame, triangulate stereo points. Then, match triangulated points in frame 1 with triangulated points in frame 2.
2. We should then be able to use these matches and corresponding depth estimates to estimate a relative pose estimate between the two frames, and add this to the graph

We should also use the matches (doesn't have to be stereo anymore) to associate landmarks IDs.


## Estimate relative pose


In [ ]:
import cv2
import torch
from lac.slam.feature_tracker import prune_features

With 3D-3D correspondences, and ICP (not working)


In [ ]:
feats_left1, feats_right1, stereo_matches1, depths1 = tracker.process_stereo(
    left_imgs[frame1], right_imgs[frame1]
)
feats_left2, feats_right2, stereo_matches2, depths2 = tracker.process_stereo(
    left_imgs[frame2], right_imgs[frame2]
)

feats_left1_triangulated = prune_features(feats_left1, stereo_matches1[:, 0])
feats_left2_triangulated = prune_features(feats_left2, stereo_matches2[:, 1])
print(f"Triangulated {len(feats_left1_triangulated['keypoints'][0])} features in image 1")
print(f"Triangulated {len(feats_left2_triangulated['keypoints'][0])} features in image 2")

triangulated_matches12 = tracker.match_feats(feats_left1_triangulated, feats_left2_triangulated)
print(f"Found {len(triangulated_matches12)} matches between the two sets of triangulated features")

In [ ]:
matched_feats1 = prune_features(feats_left1_triangulated, triangulated_matches12[:, 0])
matched_pts_left1 = matched_feats1["keypoints"][0]
depths1_matched = depths1[triangulated_matches12[:, 0]]
points_local1 = tracker.project_stereo(np.eye(4), matched_pts_left1, depths1_matched)

matched_feats2 = prune_features(feats_left2_triangulated, triangulated_matches12[:, 1])
matched_pts_left2 = matched_feats2["keypoints"][0]
depths2_matched = depths2[triangulated_matches12[:, 1]]
points_local2 = tracker.project_stereo(np.eye(4), matched_pts_left2, depths2_matched)

In [ ]:
fig = plot_3d_points(points_local1, color="red", name="Points Local 1")
fig = plot_3d_points(points_local2, fig=fig, color="blue", name="Points Local 2")
fig.show()

In [ ]:
import open3d as o3d

In [ ]:
pcd1 = o3d.geometry.PointCloud()
pcd2 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(points_local1)
pcd2.points = o3d.utility.Vector3dVector(points_local2)

# Run ICP
threshold = 0.1
icp_result = o3d.pipelines.registration.registration_icp(
    pcd1,
    pcd2,
    threshold,
    np.eye(4),
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
)

T_icp = icp_result.transformation
print("Estimated Rigid Transformation (ICP):\n", T_icp)

In [ ]:
success, M, inliers = cv2.estimateAffine3D(points_local1, points_local2)
print(f"Number of inliers: {np.sum(inliers)} out of {len(inliers)}")
print(M)

In [ ]:
est_pose1 = poses[frame1] @ np.linalg.inv(T_icp)
fig = plot_poses([poses[frame1], poses[frame2], est_pose1])
fig = plot_3d_points(
    poses[frame1][:3, 3][None, :], fig=fig, color="blue", markersize=10, name="Pose 1"
)
fig = plot_3d_points(
    poses[frame2][:3, 3][None, :], fig=fig, color="red", markersize=10, name="Pose 2"
)
fig = plot_3d_points(
    est_pose1[:3, 3][None, :], fig=fig, color="green", markersize=10, name="Estimated Pose 2"
)
fig.show()

With 2D-3D correspondences, and PnP


In [ ]:
from lac.utils.frames import (
    invert_transform_mat,
    OPENCV_TO_CAMERA_PASSIVE,
)
from lac.params import CAMERA_INTRINSICS

In [ ]:
feats_left1, feats_right1, stereo_matches1, depths1 = tracker.process_stereo(
    left_imgs[frame1], right_imgs[frame1]
)
feats_left2, feats_right2, stereo_matches2, depths2 = tracker.process_stereo(
    left_imgs[frame2], right_imgs[frame2]
)

matched_feats1 = prune_features(feats_left1, stereo_matches1[:, 0])
matched_pts_left1 = matched_feats1["keypoints"][0]
points_local1 = tracker.project_stereo(np.eye(4), matched_pts_left1, depths1)

matches12_left = tracker.match_feats(feats_left1, feats_left2)

stereo_indices = stereo_matches1[:, 0]
frame_indices = matches12_left[:, 0]

common_indices = torch.tensor(
    list(set(stereo_indices.cpu().numpy()) & set(frame_indices.cpu().numpy()))
).cuda()
frame_common = matches12_left[torch.isin(frame_indices, common_indices)]

points3D = points_local1[torch.isin(stereo_indices, common_indices).cpu().numpy()]
points2D = feats_left2["keypoints"][0][frame_common[:, 1]].cpu().numpy()

In [ ]:
success, rvec, tvec, inliers = cv2.solvePnPRansac(
    objectPoints=points3D,
    imagePoints=points2D,
    cameraMatrix=CAMERA_INTRINSICS,
    distCoeffs=None,
    flags=cv2.SOLVEPNP_ITERATIVE,
    reprojectionError=8.0,
    iterationsCount=100,
)

R, _ = cv2.Rodrigues(rvec)
T = np.hstack((R, tvec))
est_pose = np.vstack((T, [0, 0, 0, 1]))
w_T_c = invert_transform_mat(est_pose)
w_T_c[:3, :3] = w_T_c[:3, :3] @ OPENCV_TO_CAMERA_PASSIVE
rel_pose = w_T_c

In [ ]:
est_pose2 = poses[frame1] @ rel_pose
fig = plot_poses([poses[frame1], poses[frame2], est_pose2])
fig = plot_3d_points(
    poses[frame1][:3, 3][None, :], fig=fig, color="blue", markersize=10, name="Pose 1"
)
fig = plot_3d_points(
    poses[frame2][:3, 3][None, :], fig=fig, color="red", markersize=10, name="Pose 2"
)
fig = plot_3d_points(
    est_pose2[:3, 3][None, :], fig=fig, color="green", markersize=10, name="Estimated Pose 2"
)
fig.show()

In [ ]:
fig.write_html("pnp_loop_closure_poses.html")

## Add the factor


In [ ]:
from lac.slam.visual_odometry import StereoVisualOdometry
from lac.slam.loop_closure import estimate_loop_closure_pose

In [ ]:
svo = StereoVisualOdometry(cam_config)
START_FRAME = 80
svo.initialize(initial_pose, left_imgs[START_FRAME], right_imgs[START_FRAME])

In [ ]:
import gtsam
from gtsam.symbol_shorthand import B, V, X

graph = gtsam.NonlinearFactorGraph()
values = gtsam.Values()

svo_pose_sigma = 1e-2 * np.ones(6)
svo_pose_noise = gtsam.noiseModel.Diagonal.Sigmas(svo_pose_sigma)

# # Translation sigma (meters)
# sigma_t = 0.005  # 5 mm

# # Rotation sigma (radians)
# sigma_R = 0.00087  # ~0.05 degrees

# # Covariance matrix (6x6 diagonal)
# pose_noise = gtsam.noiseModel.Diagonal.Sigmas(
#     np.array([sigma_R, sigma_R, sigma_R, sigma_t, sigma_t, sigma_t])
# )

values.insert(X(0), gtsam.Pose3(initial_pose))
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(initial_pose)))

svo_poses = [initial_pose]
pose_deltas = []

END_FRAME = 4500
i = 1

for frame in tqdm(np.arange(START_FRAME + 2, END_FRAME, 2)):
    svo.track(left_imgs[frame], right_imgs[frame])
    svo_poses.append(svo.rover_pose)
    values.insert(X(i), gtsam.Pose3(svo_poses[i]))
    graph.push_back(
        gtsam.BetweenFactorPose3(X(i - 1), X(i), gtsam.Pose3(svo.pose_delta), svo_pose_noise)
    )
    i += 1

In [ ]:
frame1 = 600
frame2 = 4300
i1 = int((frame1 - START_FRAME) / 2)
i2 = int((frame2 - START_FRAME) / 2)

rel_pose = estimate_loop_closure_pose(
    tracker, left_imgs[frame1], right_imgs[frame1], left_imgs[frame2], right_imgs[frame2]
)

lc_pose_sigma = 1e-1 * np.ones(6)
lc_pose_noise = gtsam.noiseModel.Diagonal.Sigmas(svo_pose_sigma)
graph.push_back(gtsam.BetweenFactorPose3(X(i1), X(i2), gtsam.Pose3(rel_pose), lc_pose_noise))

In [ ]:
params = gtsam.LevenbergMarquardtParams()
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, values, params)
result = optimizer.optimize()

In [ ]:
opt_poses = []
for i in range(len(svo_poses)):
    opt_poses.append(result.atPose3(X(i)).matrix())

In [ ]:
fig = plot_poses(poses[:END_FRAME], no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, no_axes=True, fig=fig, color="red", name="VO")
fig = plot_poses(opt_poses, no_axes=True, fig=fig, color="green", name="Optimized")
fig.show()

# Loop closure detection

- Position based: assuming drift is not too high, attempt to match frames which have similar pose estimates.
- Visual based: compute bag-of-words (BoW) descriptors for each frame and form a visual vocabulary. Then, match frames based on the BoW descriptors.

We can also program in some purposeful loop closure maneuvers, like turn and look at the lander
every N frames.
